In [1]:
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams["savefig.format"]="png"
#plt.rcParams['figure.figsize'] = [16,9]
np.random.seed(0)

In [2]:
#Import Data
names=["Class", "Sex", "Age", "Sibling and Spouses", "Parents and Children" , "Fare", "Survivors"]
data = np.loadtxt("titanic-train.txt")
test_data = np.loadtxt("titanic-test.txt")
#print(test_data.shape)

for i in range(7):
    plt.title(names[i])
    plt.xlabel(names[i])
    plt.ylabel("Relative Frequency")
    plt.grid(1)
    plt.hist(data[:,i], weights=(np.zeros_like(data[:,i])+1./data[:,i].size)*100)
    #plt.show()
    plt.savefig("images/"+names[i])
    plt.clf()

cl=data[:,0]
sex=data[:,1]
surv=data[:,6]
a=np.zeros(3)
b=np.zeros(2)
for i in range(cl.size):
    if sex[i]==0 and surv[i]==1:
        b[0]+=1
    if sex[i]==1 and surv[i]==1:
        b[1]+=1
    if cl[i]==1 and surv[i]==1:
        a[0]+=1
    elif cl[i]==2 and surv[i]==1:
        a[1]+=1
    elif cl[i]==3 and surv[i]==1:
        a[2]+=1
        
#a[2]=cl.size-a[0]-a[1]
labc=[1,2,3]
labs=["Male", "Female"]
a/=a.sum()/100
b*=100/b.sum()
plt.bar(labc,a)
plt.grid(1)
plt.title("Class Survival Empirical Probability")
plt.xlabel("Class")
plt.ylabel("Surviving Probability")
#plt.show()
plt.savefig("images/"+"class_emp")
plt.clf()

plt.bar(labs,b)
plt.grid(1)
plt.title("Sex Survival Empirical Probability")
plt.xlabel("Sex")
plt.ylabel("Surviving Probability")
#plt.show()
plt.savefig("images/"+"sex_emp")
plt.clf()

#Plot Data Correlation
for i in range(6):
    for j in range(i+1,6):
        print(i,j)
        plt.title(names[i] + " vs. " + names[j])
        plt.xlabel(names[i])
        plt.ylabel(names[j])
        plt.scatter(data[:,i], data[:,j], c=data[:,6], alpha=0.5)
        plt.grid(1)
        #plt.show()
        plt.savefig("images/"+names[i] + " vs. " + names[j]+".png")
        plt.clf()

0 1
0 2
0 3
0 4
0 5
1 2
1 3
1 4
1 5
2 3
2 4
2 5
3 4
3 5
4 5


<Figure size 432x288 with 0 Axes>

In [12]:
#Logistic Regression Functions

def sigmoid(z):
    return 1./(1.+np.exp(-z))

def logreg_inference(x,w,b):
    return sigmoid(x@w+b)

def cross_entropy(p,y):
    return (-y*np.log(p)-(1-y)*np.log(1-p)).mean()

def logreg_train(x, y, epochs, lr,l, x_t,y_t):
    s_loss=np.zeros(epochs)
    s_tloss=np.zeros(epochs)
    s_acc=np.zeros(epochs)
    s_tacc=np.zeros(epochs)
    m,n=x.shape
    w=np.zeros(n)
    b=0
    n=0
    print("Steps  Loss  Accuracy")
    for step in range(epochs):
        p=logreg_inference(x,w,b)
        s_loss[step]=cross_entropy(p,y)
        predictions=np.asarray(p>0.5)
        s_acc[step]=np.asarray(predictions==y).mean()
        pt=logreg_inference(x_t,w,b)
        tpred=np.asarray(pt>0.5)
        s_tloss[step]= cross_entropy(pt,y_t)
        s_tacc[step]= np.asarray(tpred==y_t).mean()
        if step%10000==0:
            #print(step, loss)
            print(step, s_loss[step], s_tloss[step], s_acc[step]*100)
        grad_b=(p-y).mean()#+2*l*(b)
        grad_w=(x.T@(p-y))/m+2*l*(w)
        b-=lr*grad_b
        w-=lr*grad_w
        '''
        oldp=logreg_inference(x,w,b)
        oldloss=cross_entropy(oldp,y)
        oldpredictions=(oldp>0.5)
        oldaccuracy=(oldpredictions==y).mean()
        '''
    np.save("loss", s_loss)
    np.save("loss_test", s_tloss)
    np.save("accuracy", s_acc)
    np.save("accuracy_test",s_tacc)
    return w,b

#Assign Data and Labels
x=data[:,:6]
y=data[:,6]
x_t=test_data[:,:6]
y_t=test_data[:,6]

In [15]:
#Training
w,b = logreg_train(x,y, int(2*1e5),0.001,0.01, x_t,y_t)
p=logreg_inference(x,w,b)
predictions=(p>0.5)
accuracy=(predictions==y).mean()
print("Training Set Accuracy: ", accuracy*100)

Steps  Loss  Accuracy
0 0.6931471805599453 0.6931471805599453 62.112676056338024
10000 0.5137066609659817 0.5455064355601694 74.92957746478874
20000 0.4765258933567252 0.514912502870386 79.71830985915493
30000 0.45973121632045144 0.5028019499971507 80.70422535211267
40000 0.45135616844164683 0.4976128587020684 79.85915492957747
50000 0.44667262957334886 0.4950847604465594 79.71830985915493
60000 0.44376223578255725 0.49371580950834204 79.85915492957747
70000 0.4417895599475514 0.49291395304494845 80.0
80000 0.44034970242890437 0.4923993370719554 80.0
90000 0.4392332358550912 0.4920371699120373 80.14084507042254
100000 0.438325966176899 0.4917616929361822 80.14084507042254
110000 0.43756250980698436 0.49154040462471116 80.28169014084507
120000 0.4369035835654725 0.49135691341894516 80.28169014084507
130000 0.4363244173009672 0.49120262950919397 80.14084507042254
140000 0.4358086247646049 0.49107272021348064 80.14084507042254
150000 0.43534485219763386 0.4909641452328547 80.0
160000 0.43

In [5]:
s_loss=np.load("loss.npy")
s_tloss=np.load("loss_test.npy")
s_acc=np.load("accuracy.npy")
s_tacc=np.load("accuracy_test.npy")

plt.plot(s_loss, label="Training Loss")
plt.plot(s_tloss, label="Test Loss")
plt.grid(1)
plt.legend()
plt.title("Loss Functions")
plt.xlabel("Steps")
plt.ylabel("Loss Value")
plt.savefig("images/"+"losses")
plt.clf()
#plt.show()

plt.plot(s_acc, label="Training Accuracy")
plt.plot(s_tacc, label="Test Accuracy")
plt.grid(1)
plt.legend()
plt.title("Accuracy")
plt.xlabel("Steps")
plt.ylabel("Accuracy Value")
plt.savefig("images/"+"acc")
plt.clf()
#plt.show()

/home/daniele/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  del sys.path[0]
/home/daniele/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.


<Figure size 432x288 with 0 Axes>

In [6]:
#Results on the Training Set
print(w,b)

for i in range(6):
    for j in range(i+1,6):
        print(i,j)
        plt.title(names[i] + " vs. " + names[j])
        plt.xlabel(names[i])
        plt.ylabel(names[j])
        decision_bound=-w[i]/w[j]*data[:,i]-b/w[j]
        plt.plot(data[:,i], decision_bound, label="Decision Bound")
        plt.scatter(data[:,i], data[:,j], c=data[:,6], alpha=0.5, label="Data")
        plt.savefig("images/"+"bound_"+ names[i] + " vs. " + names[j]+".png")
        plt.clf()
        #plt.show()

[-0.97398596  2.76847753 -0.03290549 -0.3023238  -0.09865615  0.00367901] 1.5634614668927844
0 1
0 2
0 3
0 4
0 5
1 2
1 3
1 4
1 5
2 3
2 4
2 5
3 4
3 5
4 5


<Figure size 432x288 with 0 Axes>

In [7]:
#Guess a Survival Probability
guess_data=np.zeros((1,6))

for i in range(5):
    guess_data[0][i]=np.random.randint(data[:,i].min(), data[:,i].max())

guess_data[0][5]=(data[:,5].max()-data[:,5].min())*np.random.rand()+data[:,5].min()
print(guess_data)

prob_guess=logreg_inference(guess_data,w,b)
print("Random Guess: ", prob_guess*100)

#Educated Guess

guess_data[0][0]=1
guess_data[0][1]=1
edn=int(1e5)
ed_g=np.zeros(edn)
ed_d=np.zeros(edn)

for i in range(edn):
    guess_data[0][0]=1
    guess_data[0][1]=1
    for j in range(2,5):
        guess_data[0][j]=np.random.randint(data[:,j].min(), data[:,j].max())
    guess_data[0][3]=1
    ed_g[i]=logreg_inference(guess_data,w,b)
    guess_data[0][0]=3
    guess_data[0][1]=0
    guess_data[0][3]=8
    ed_d[i]=logreg_inference(guess_data,w,b)

bins=np.zeros(10)
bind=np.zeros(10)
for i in range(edn):
    for j in range(10):
        if ed_g[i]*100<(j+1)*10 and ed_g[i]*100>j*10:
            bins[j]+=1
        if ed_d[i]*100<(j+1)*10 and ed_d[i]*100>j*10:
            bind[j]+=1

pl=np.zeros(10)
for i in range(10):
    pl[i]=(10*i+5)
print(pl)
plt.bar(pl,bins/bins.sum(), width=8, label="Women, Alone, 1 Class")
plt.bar(pl,bind/bind.sum(), width=8, label="Man, Not Alone, 3 Class")
plt.title(str(edn)+ " Educated Guesses")
plt.legend()
plt.grid()
plt.xticks(list(pl), pl)
plt.savefig("images/"+"ed_guess")
plt.clf()
#plt.show()
prob_guess=logreg_inference(guess_data,w,b)
print("Educated Guess: ",prob_guess*100)

[[  1.           0.          47.           5.           0.
  308.81327825]]
Random Guess:  [20.87338058]
[ 5. 15. 25. 35. 45. 55. 65. 75. 85. 95.]
Educated Guess:  [1.31364633]


<Figure size 432x288 with 0 Axes>

In [8]:
#Evaluate the model on test data
p=logreg_inference(x_t,w,b)
predictions=(p>0.5)
accuracy=(predictions==y_t).mean()
print("Test Accuracy: ", accuracy*100)

for i in range(6):
    for j in range(i+1,6):
        print(i,j)
        plt.title(names[i] + " vs. " + names[j])
        plt.xlabel(names[i])
        plt.ylabel(names[j])
        decision_bound=-w[i]/w[j]*test_data[:,i]-b/w[j]
        plt.plot(test_data[:,i], decision_bound, label="Decision Bound")
        plt.scatter(test_data[:,i], test_data[:,j], c=test_data[:,6], alpha=0.5, label="Data")
        plt.savefig("images/"+"test_bound_"+names[i] + " vs. " + names[j]+".png")
        plt.clf()
        #plt.show()

Test Accuracy:  79.09604519774011
0 1
0 2
0 3
0 4
0 5
1 2
1 3
1 4
1 5
2 3
2 4
2 5
3 4
3 5
4 5


<Figure size 432x288 with 0 Axes>